# 基于CNN和EEMD的脑电信号麻醉深度预测
复现论文：[Depth of anesthesia prediction via EEG signals using convolutionalneural network and ensemble empirical mode decomposition](https://bura.brunel.ac.uk/handle/2438/22883)

此论文思想是通过EEMD将脑电信号进行分解，选取前4个分解结果，通过STFT（短时傅里叶变换）生成频谱图并拼合在一起。然后通过CNN进行分类。

由于作者没有给出他自己的网络的具体参数，故此项目只是按照他的思想进行复现。

   
      eemd1.py 是原始脑电信号数据处理成频谱图的代码
      eemd2.py
      resize.py 是图片转换成128x128大小的代码

解压data中的img128.tar到主目录，然后用paddlex划分数据集
```
    paddlex --split_dataset --format SEG --dataset_dir img128 --val_value 0.2 --test_value 0.1
```

## 使用ResNet50进行训练

In [1]:
import os
import cv2
import numpy as np
import paddle
from paddle.vision.transforms import Normalize
from paddle.io import Dataset

class MyDataset(Dataset):
    """
    步骤一：继承 paddle.io.Dataset 类
    """
    def __init__(self, data_dir, label_path, transform=None):
        """
        步骤二：实现 __init__ 函数，初始化数据集，将样本和标签映射到列表中
        """
        super(MyDataset, self).__init__()
        self.data_list = []
        with open(label_path,encoding='utf-8') as f:
            for line in f.readlines():
                image_path, label = line.strip().split()
                image_path = os.path.join(data_dir, image_path)
                self.data_list.append([label,image_path])
        # 传入定义好的数据处理方法，作为自定义数据集类的一个属性
        self.transform = transform

    def __getitem__(self, index):
        """
        步骤三：实现 __getitem__ 函数，定义指定 index 时如何获取数据，并返回单条数据（样本数据、对应的标签）
        """
        # 根据索引，从列表中取出一个图像
        label,image_path = self.data_list[index]
        # 读图
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        # 飞桨训练时内部数据格式默认为float32，将图像数据格式转换为 float32
        image = image.astype('float32')
        image = image.transpose([2,0,1])
        # 应用数据处理方法到图像上
        if self.transform is not None:
            image = self.transform(image)
        # CrossEntropyLoss要求label格式为int，将Label格式转换为 int
        label = int(label)
        # 返回图像和对应标签
        return image, label

    def __len__(self):
        """
        步骤四：实现 __len__ 函数，返回数据集的样本总数
        """
        return len(self.data_list)

# 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')
# 打印数据集样本数        
train_custom_dataset = MyDataset('img128','img128/train_list.txt', transform)
val_custom_dataset = MyDataset('img128','img128/val_list.txt', transform)
print('train_custom_dataset images: ',len(train_custom_dataset), 'val_custom_dataset images: ',len(val_custom_dataset))


train_custom_dataset images:  23826 val_custom_dataset images:  6805


In [1]:
import paddle
import paddle.nn as nn
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.transforms import Transpose
from paddle.utils.download import get_weights_path_from_url

__all__ = []

model_urls = {
    'resnet50': ('https://paddle-hapi.bj.bcebos.com/models/resnet50.pdparams',
                 'ca6f485ee1ab0492d38f323885b0ad80'),
}
class BasicBlock(nn.Layer):
    expansion = 1

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 downsample=None,
                 groups=1,
                 base_width=64,
                 dilation=1,
                 norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2D

        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock")

        self.conv1 = nn.Conv2D(
            inplanes, planes, 3, padding=1, stride=stride, bias_attr=False)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2D(planes, planes, 3, padding=1, bias_attr=False)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class BottleneckBlock(nn.Layer):

    expansion = 4

    def __init__(self,
                 inplanes,
                 planes,
                 stride=1,
                 downsample=None,
                 groups=1,
                 base_width=64,
                 dilation=1,
                 norm_layer=None):
        super(BottleneckBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2D
        width = int(planes * (base_width / 64.)) * groups

        self.conv1 = nn.Conv2D(inplanes, width, 1, bias_attr=False)
        self.bn1 = norm_layer(width)

        self.conv2 = nn.Conv2D(
            width,
            width,
            3,
            padding=dilation,
            stride=stride,
            groups=groups,
            dilation=dilation,
            bias_attr=False)
        self.bn2 = norm_layer(width)

        self.conv3 = nn.Conv2D(
            width, planes * self.expansion, 1, bias_attr=False)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Layer):
    def __init__(self, block, depth, num_classes=1000, with_pool=True):
        super(ResNet, self).__init__()
        layer_cfg = {
            50: [3, 4, 6, 3]
        }
        layers = layer_cfg[depth]
        self.num_classes = num_classes
        self.with_pool = with_pool
        self._norm_layer = nn.BatchNorm2D

        self.inplanes = 64
        self.dilation = 1

        self.conv1 = nn.Conv2D(
            3,
            self.inplanes,
            kernel_size=7,
            stride=2,
            padding=3,
            bias_attr=False)
        self.bn1 = self._norm_layer(self.inplanes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2D(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        if with_pool:
            self.avgpool = nn.AdaptiveAvgPool2D((1, 1))

        if num_classes > 0:
            self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2D(
                    self.inplanes,
                    planes * block.expansion,
                    1,
                    stride=stride,
                    bias_attr=False),
                norm_layer(planes * block.expansion), )

        layers = []
        layers.append(
            block(self.inplanes, planes, stride, downsample, 1, 64,
                  previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        if self.with_pool:
            x = self.avgpool(x)

        if self.num_classes > 0:
            x = paddle.flatten(x, 1)
            x = self.fc(x)

        return x


def _resnet(arch, Block, depth, pretrained, **kwargs):
    model = ResNet(Block, depth, **kwargs)
    if pretrained:
        assert arch in model_urls, "{} model do not have a pretrained model now, you should set pretrained=False".format(
            arch)
        weight_path = get_weights_path_from_url(model_urls[arch][0],
                                                model_urls[arch][1])

        param = paddle.load(weight_path)
        model.set_dict(param)

    return model



def resnet50(pretrained=False, **kwargs):
    return _resnet('resnet50', BottleneckBlock, 50, pretrained, **kwargs)

In [6]:
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.transforms import Transpose
paddle.set_device('gpu')
# 调用resnet50模型
res50_model = paddle.Model(resnet50(pretrained=False, num_classes=3))

# 定义优化器
optimizer = Momentum(learning_rate=0.01,
                     momentum=0.9,
                     weight_decay=L2Decay(1e-4),
                     parameters=res50_model.parameters())
# 进行训练前准备
res50_model.prepare(optimizer, CrossEntropyLoss(), Accuracy(topk=(1, 5)))
# 启动训练
res50_model.fit(train_custom_dataset,
          val_custom_dataset,
          epochs=50,
          batch_size=128,
          save_dir="./output-ResNet50",
          num_workers=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50
step  10/187 - loss: 3.6585 - acc_top1: 0.3930 - acc_top5: 1.0000 - 423ms/step
step  20/187 - loss: 2.7920 - acc_top1: 0.4195 - acc_top5: 1.0000 - 439ms/step
step  30/187 - loss: 0.9385 - acc_top1: 0.4367 - acc_top5: 1.0000 - 419ms/step
step  40/187 - loss: 1.0407 - acc_top1: 0.4652 - acc_top5: 1.0000 - 406ms/step
step  50/187 - loss: 1.0900 - acc_top1: 0.4909 - acc_top5: 1.0000 - 398ms/step
step  60/187 - loss: 1.5024 - acc_top1: 0.5038 - acc_top5: 1.0000 - 397ms/step
step  70/187 - loss: 0.9898 - acc_top1: 0.5155 - acc_top5: 1.0000 - 404ms/step
step  80/187 - loss: 0.9101 - acc_top1: 0.5327 - acc_top5: 1.0000 - 399ms/step
step  90/187 - loss: 0.7554 - acc_top1: 0.5493 - acc_top5: 1.0000 - 396ms/step
step 100/187 - loss: 0.7045 - acc_top1: 0.5624 - acc_top5: 1.0000 - 393ms/step
step 110/187 - loss: 0.6625 - acc_top1: 0.5723 - acc_top5: 1.0000 - 391ms/step
step 120/1

In [7]:
test_custom_dataset = MyDataset('img128','img128/test_list.txt', transform)
eval_result = res50_model.evaluate(test_custom_dataset, verbose=1)
print(eval_result)

Eval begin...
step 3402/3402 [==============================] - loss: 1.1074e-04 - acc_top1: 0.7513 - acc_top5: 1.0000 - 18ms/step      
Eval samples: 3402
{'loss': [0.000110739296], 'acc_top1': 0.7513227513227513, 'acc_top5': 1.0}


评估可得准确率为75%

In [ ]:
test_result = res50_model.predict(test_custom_dataset)


In [25]:
print(len(test_result))
print(test_result[0][11])

# 从测试集中取出一张图片
img ,label = test_custom_dataset[11]
# 打印推理结果，这里的argmax函数用于取出预测值中概率最高的一个的下标，作为预测标签
pred_label = test_result[0][11].argmax()
print('true label: {}, pred label: {}'.format(label, pred_label))

1
[[ 22.578781 -18.686363  -5.079281]]
true label: 0, pred label: 0


## 使用论文作者自己定义的网络
| Layer Type|Size|
|--|--|
| Image |128\*128\*3|
|Conv-1|55\*55\*96|
|Conv-2|27\*27\*32|
|Conv-3|13\*13\*64|
|Conv-4|11\*11\*128|
|Conv-5|5\*5\*128|
|Conv-6|5\*5\*256|
|FC|1024|
|Ouput|3|

由于作者没有具体参数，故卷积核大小，激活函数种类，padding大小，stride大小都无法确定，只能猜测。故准确率和作者有较大区别。

In [1]:
import os
import cv2
import numpy as np
import paddle
from paddle.vision.transforms import Normalize
from paddle.io import Dataset

class MyDataset(Dataset):
    """
    步骤一：继承 paddle.io.Dataset 类
    """
    def __init__(self, data_dir, label_path, transform=None):
        """
        步骤二：实现 __init__ 函数，初始化数据集，将样本和标签映射到列表中
        """
        super(MyDataset, self).__init__()
        self.data_list = []
        with open(label_path,encoding='utf-8') as f:
            for line in f.readlines():
                image_path, label = line.strip().split()
                image_path = os.path.join(data_dir, image_path)
                self.data_list.append([label,image_path])
        # 传入定义好的数据处理方法，作为自定义数据集类的一个属性
        self.transform = transform

    def __getitem__(self, index):
        """
        步骤三：实现 __getitem__ 函数，定义指定 index 时如何获取数据，并返回单条数据（样本数据、对应的标签）
        """
        # 根据索引，从列表中取出一个图像
        label,image_path = self.data_list[index]
        # 读图
        image = cv2.imread(image_path)
        image = cv2.resize(image,(128, 128))
        # 飞桨训练时内部数据格式默认为float32，将图像数据格式转换为 float32
        image = image.astype('float32')
        image = image.transpose([2,0,1])
        # 应用数据处理方法到图像上
        if self.transform is not None:
            image = self.transform(image)
        # CrossEntropyLoss要求label格式为int，将Label格式转换为 int
        label = int(label)
        # 返回图像和对应标签
        return image, label

    def __len__(self):
        """
        步骤四：实现 __len__ 函数，返回数据集的样本总数
        """
        return len(self.data_list)

# 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')
# 打印数据集样本数        
train_custom_dataset = MyDataset('img128','img128/train_list.txt', transform)
val_custom_dataset = MyDataset('img128','img128/val_list.txt', transform)
print('train_custom_dataset images: ',len(train_custom_dataset), 'val_custom_dataset images: ',len(val_custom_dataset))

train_custom_dataset images:  23826 val_custom_dataset images:  6805


In [8]:
from paddle import nn
custom_net=nn.Sequential(
    nn.Conv2D(3, 96, 20, stride=2, padding=0),
    nn.LeakyReLU(),
    nn.Conv2D(96, 32, 4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2D(32, 64, 4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2D(64, 128, 4, stride=2, padding=0),
    nn.LeakyReLU(),
    nn.Conv2D(128, 256, 3, stride=1, padding=1),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(6400,1024),
    nn.Sigmoid(),
    nn.Linear(1024,3),
)
paddle.summary(custom_net,(1, 3, 128, 128))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-6      [[1, 3, 128, 128]]    [1, 96, 55, 55]        115,296    
  LeakyReLU-8    [[1, 96, 55, 55]]     [1, 96, 55, 55]           0       
   Conv2D-7      [[1, 96, 55, 55]]     [1, 32, 27, 27]        49,184     
  LeakyReLU-9    [[1, 32, 27, 27]]     [1, 32, 27, 27]           0       
   Conv2D-8      [[1, 32, 27, 27]]     [1, 64, 13, 13]        32,832     
 LeakyReLU-10    [[1, 64, 13, 13]]     [1, 64, 13, 13]           0       
   Conv2D-9      [[1, 64, 13, 13]]      [1, 128, 5, 5]        131,200    
 LeakyReLU-11     [[1, 128, 5, 5]]      [1, 128, 5, 5]           0       
   Conv2D-10      [[1, 128, 5, 5]]      [1, 256, 5, 5]        295,168    
 LeakyReLU-12     [[1, 256, 5, 5]]      [1, 256, 5, 5]           0       
   Flatten-2      [[1, 256, 5, 5]]        [1, 6400]              0       
   Linear-3         [[1, 6400]]     

{'total_params': 7181379, 'trainable_params': 7181379}

In [9]:
from paddle.optimizer import Momentum,Adam
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
paddle.set_device('gpu')
custom_model=paddle.Model(custom_net)
# 定义优化器
optimizer = Adam(learning_rate=0.01,
        parameters=custom_model.parameters())
# 进行训练前准备
custom_model.prepare(optimizer, CrossEntropyLoss(), Accuracy(topk=(1, 5)))
# 启动训练
custom_model.fit(train_custom_dataset,
          val_custom_dataset,
          epochs=20,
          batch_size=256,
          save_dir="./output-custom")

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20
step 10/94 - loss: 3.8181 - acc_top1: 0.3941 - acc_top5: 1.0000 - 496ms/step
step 20/94 - loss: 2.1430 - acc_top1: 0.4227 - acc_top5: 1.0000 - 472ms/step
step 30/94 - loss: 1.2670 - acc_top1: 0.4257 - acc_top5: 1.0000 - 456ms/step
step 40/94 - loss: 1.0965 - acc_top1: 0.4122 - acc_top5: 1.0000 - 460ms/step
step 50/94 - loss: 1.0385 - acc_top1: 0.4146 - acc_top5: 1.0000 - 464ms/step
step 60/94 - loss: 1.0966 - acc_top1: 0.4234 - acc_top5: 1.0000 - 459ms/step
step 70/94 - loss: 1.0512 - acc_top1: 0.4315 - acc_top5: 1.0000 - 460ms/step
step 80/94 - loss: 0.9993 - acc_top1: 0.4341 - acc_top5: 1.0000 - 458ms/step
step 90/94 - loss: 0.9721 - acc_top1: 0.4357 - acc_top5: 1.0000 - 457ms/step
step 94/94 - loss: 1.1175 - acc_top1: 0.4384 - acc_top5: 1.0000 - 452ms/step
save checkpoint at /home/aistudio/output-custom/0
Eval begin...
step 10/27 - loss: 1.6816 - acc_top1: 0.0000e

In [10]:
test_custom_dataset = MyDataset('img128','img128/test_list.txt', transform)
eval_result = custom_model.evaluate(test_custom_dataset, verbose=1)
print(eval_result)

Eval begin...
step 3402/3402 [==============================] - loss: 0.4705 - acc_top1: 0.6605 - acc_top5: 1.0000 - 42ms/step          
Eval samples: 3402
{'loss': [0.47046927], 'acc_top1': 0.6604938271604939, 'acc_top5': 1.0}


可得准确率为66%

## 使用ResNet152

In [2]:
import os
import cv2
import numpy as np
import paddle
from paddle.vision.transforms import Normalize
from paddle.io import Dataset

class MyDataset(Dataset):
    """
    步骤一：继承 paddle.io.Dataset 类
    """
    def __init__(self, data_dir, label_path, transform=None):
        """
        步骤二：实现 __init__ 函数，初始化数据集，将样本和标签映射到列表中
        """
        super(MyDataset, self).__init__()
        self.data_list = []
        with open(label_path,encoding='utf-8') as f:
            for line in f.readlines():
                image_path, label = line.strip().split()
                image_path = os.path.join(data_dir, image_path)
                self.data_list.append([label,image_path])
        # 传入定义好的数据处理方法，作为自定义数据集类的一个属性
        self.transform = transform

    def __getitem__(self, index):
        """
        步骤三：实现 __getitem__ 函数，定义指定 index 时如何获取数据，并返回单条数据（样本数据、对应的标签）
        """
        # 根据索引，从列表中取出一个图像
        label,image_path = self.data_list[index]
        # 读图
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        # 飞桨训练时内部数据格式默认为float32，将图像数据格式转换为 float32
        image = image.astype('float32')
        image = image.transpose([2,0,1])
        # 应用数据处理方法到图像上
        if self.transform is not None:
            image = self.transform(image)
        # CrossEntropyLoss要求label格式为int，将Label格式转换为 int
        label = int(label)
        # 返回图像和对应标签
        return image, label

    def __len__(self):
        """
        步骤四：实现 __len__ 函数，返回数据集的样本总数
        """
        return len(self.data_list)

# 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')
# 打印数据集样本数        
train_custom_dataset = MyDataset('img128','img128/train_list.txt', transform)
val_custom_dataset = MyDataset('img128','img128/val_list.txt', transform)
print('train_custom_dataset images: ',len(train_custom_dataset), 'val_custom_dataset images: ',len(val_custom_dataset))

train_custom_dataset images:  23826 val_custom_dataset images:  6805


In [3]:
from paddle.optimizer import Momentum,Adam
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.models import resnet152

paddle.set_device('gpu')
res152_model=paddle.Model(resnet152(pretrained=False, num_classes=3))
# 定义优化器
optimizer = Adam(learning_rate=0.01,
        parameters=res152_model.parameters())
# 进行训练前准备
res152_model.prepare(optimizer, CrossEntropyLoss(), Accuracy(topk=(1, 5)))
# 启动训练
res152_model.fit(train_custom_dataset,
          val_custom_dataset,
          epochs=20,
          batch_size=64,
          save_dir="./output-ResNet152",
          num_workers=1)

W0824 11:50:32.423933   167 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0824 11:50:32.428511   167 gpu_resources.cc:91] device: 0, cuDNN Version: 7.6.


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step  10/373 - loss: 1.6064 - acc_top1: 0.4047 - acc_top5: 1.0000 - 662ms/step
step  20/373 - loss: 1.1934 - acc_top1: 0.3773 - acc_top5: 1.0000 - 558ms/step
step  30/373 - loss: 1.2960 - acc_top1: 0.4224 - acc_top5: 1.0000 - 525ms/step
step  40/373 - loss: 1.2399 - acc_top1: 0.4379 - acc_top5: 1.0000 - 509ms/step
step  50/373 - loss: 1.4175 - acc_top1: 0.4512 - acc_top5: 1.0000 - 498ms/step
step  60/373 - loss: 1.0961 - acc_top1: 0.4576 - acc_top5: 1.0000 - 495ms/step
step  70/373 - loss: 1.0101 - acc_top1: 0.4667 - acc_top5: 1.0000 - 491ms/step
step  80/373 - loss: 0.9548 - acc_top1: 0.4721 - acc_top5: 1.0000 - 486ms/step
step  90/373 - loss: 0.9334 - acc_top1: 0.4795 - acc_top5: 1.0000 - 483ms/step
step 100/373 - loss: 0.9634 - acc_top1: 0.4800 - acc_top5: 1.0000 - 481ms/step
step 110/373 - loss: 1.2586 - acc_top1: 0.4832 - acc_top5: 1.0000 - 478ms/step
step 120/373 - loss: 1.0161 - acc_top1: 0.4854 - acc_top5: 1.0000 - 477ms/step
step 130/373 - loss: 0.9903 - acc_top1: 0.4897 - acc

In [4]:
test_custom_dataset = MyDataset('img128','img128/test_list.txt', transform)
eval_result = res152_model.evaluate(test_custom_dataset, verbose=1)
print(eval_result)

Eval begin...
step 3402/3402 [==============================] - loss: 0.5630 - acc_top1: 0.7551 - acc_top5: 1.0000 - 52ms/step          
Eval samples: 3402
{'loss': [0.5630327], 'acc_top1': 0.7551440329218106, 'acc_top5': 1.0}


准确率为76%